In [387]:
import pandas as pd

from scipy.stats import normaltest

import scipy.stats as stats

In [388]:
data_con = pd.read_csv('../data/processed/Data_continuous.csv', index_col=0)
data_cat = pd.read_csv('../data/processed/Data_categorical.csv', index_col=0)
data_lab = pd.read_csv('../data/processed/Data_label.csv', index_col=0)
data_con_lab = pd.concat([data_con, data_lab], axis=1)
data_cat_lab = pd.concat([data_cat, data_lab], axis=1)

Mann-Whitney U (Wilcoxon rank-sum test)
Given two independent samples \( X_1, X_2, \ldots, X_m \) and \( Y_1, Y_2, \ldots, Y_n \), with ranks \( R(X_i) \) and \( R(Y_j) \) in the combined sample:

1. Calculate the U statistics:
   $$
   U_X = m \cdot n + \frac{m(m+1)}{2} - \sum_{i=1}^m R(X_i)
   $$
   $$
   U_Y = m \cdot n + \frac{n(n+1)}{2} - \sum_{j=1}^n R(Y_j)
   $$
   $$
   U = \min(U_X, U_Y)
   $$

2. Calculate the p-value:
   Based on the U statistic and sample sizes, refer to the Mann-Whitney U distribution table or use normal approximation for large samples.


In [389]:
# Chi-square test for categorical variables
def chi_square_test(df):
    binary_var = df.columns[-1]
    categorical_vars = df.columns[:-1]
    
    results = []
    
    for var in categorical_vars:
        # -1 for definition of fixation and discordance
        df_clean = df[(df[var].notna()) & (df[binary_var].notna()) & (df[var] != -1) & (df[binary_var] != -1)]

        contingency_table = pd.crosstab(df_clean[var], df_clean[binary_var])

        chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
        results.append({
            'Variable': var,
            'Chi2 Statistic': chi2,
            'P Value': p,
            'Degrees of Freedom': dof,
        })
    
    results_df = pd.DataFrame(results)
    return results_df

In [390]:
# Normal distribution testing is used to determine which method to use
def check_normality(df):
    rows = []

    for col in df.columns[:-1]:
        column_data = df[col].dropna()
        _, p_value = normaltest(column_data)

        is_normal = 'Yes' if p_value >= 0.05 else 'No'
        rows.append({
            'Metric': col,
            'Mean': column_data.mean(),
            'Median': column_data.median(),
            'Min': column_data.min(),
            'Max': column_data.max(),
            'Normality p-value': p_value,
            'Is Normal?': is_normal 
        })

    result_df = pd.DataFrame(rows)
    return result_df

In [391]:
# Continuous variables were tested using the Mann-Whitney U test
def mann_whitney_test(df):
    binary_var = 'FBTP/NFBTP'
    continuous_vars = df.columns.difference([binary_var])

    results = []

    for var in continuous_vars:
        group0 = df[df[binary_var] == 0][var].dropna()
        group1 = df[df[binary_var] == 1][var].dropna()
        # Mann-Whitney U test
        u_statistic, p_value = stats.mannwhitneyu(group0, group1, alternative='two-sided')

        results.append({
            'Metric': var,
            'U Statistic': u_statistic,
            'P Value': p_value
        })

    results_df = pd.DataFrame(results)
    return results_df

# Imaging parameters

## I. PSMA imaging parameters analysis

In [392]:
psma_filtered_metrics = ['Max', 'Min', 'Mean', 'Peak', 'Std. dev']
psma_cols = [col for col in data_con_lab.columns if ('PSMA' in col and any(metric in col for metric in psma_filtered_metrics)) or col in ['FBTP/NFBTP',]]

PSMA_imaging = data_con_lab[psma_cols]

In [393]:
# Defination of orders
primary_order = ['SalivaryGland', 'Liver', 'Kidney', 'Bone+','Bone-', 'LymphNode+', 'LymphNode-', 'Prostate']
secondary_order = ['Max', 'Min', 'Mean', 'Peak', 'Std. dev']

def primary_sort_key(metric, order):
    for i, key in enumerate(order):
        if key in metric:
            return i
    return len(order)

def secondary_sort_key(metric, order):
    for i, key in enumerate(order):
        if key in metric:
            return i
    return len(order)

### 1. Normal distribution test

In [394]:
normality_result_df_psma = check_normality(PSMA_imaging)

normality_result_df_psma['primary_sort_key'] = normality_result_df_psma['Metric'].apply(lambda x: primary_sort_key(x, primary_order))
normality_result_df_psma['secondary_sort_key'] = normality_result_df_psma['Metric'].apply(lambda x: secondary_sort_key(x, secondary_order))

normality_result_df_psma = normality_result_df_psma.sort_values(by=['primary_sort_key', 'secondary_sort_key']).drop(columns=['primary_sort_key', 'secondary_sort_key'])

In [395]:
normality_result_df_psma

,Metric,Mean,Median,Min,Max,Normality p-value,Is Normal?
7,Max: g/ml_PSMA_SalivaryGland,14.2912,13.66,3.85,24.43,8.796143e-01,Yes
23,Min: g/ml_PSMA_SalivaryGland,6.0192,5.77,1.63,10.28,8.851870e-01,Yes
15,Mean: g/ml_PSMA_SalivaryGland,9.3348,8.81,2.66,16.70,6.804944e-01,Yes
31,Peak: g/ml_PSMA_SalivaryGland,12.0352,11.18,3.36,21.26,7.731037e-01,Yes
39,Std. dev: g/ml_PSMA_SalivaryGland,1.9704,1.81,0.56,3.26,9.174261e-01,Yes
3,Max: g/ml_PSMA_Liver,7.4092,6.76,4.59,12.05,1.223000e-01,Yes
19,Min: g/ml_PSMA_Liver,3.2936,3.30,1.96,5.55,3.284241e-01,Yes
11,Mean: g/ml_PSMA_Liver,4.6484,4.43,2.69,7.35,4.237251e-01,Yes
27,Peak: g/ml_PSMA_Liver,4.8248,4.82,0.00,8.47,5.969204e-02,Yes
35,Std. dev: g/ml_PSMA_Liver,0.6304,0.52,0.33,2.04,6.804956e-07,No


### 2. Significant differences test

In [396]:
difference_results_df_psma = mann_whitney_test(PSMA_imaging)
difference_results_df_psma['primary_sort_key'] = difference_results_df_psma['Metric'].apply(lambda x: primary_sort_key(x, primary_order))
difference_results_df_psma['secondary_sort_key'] = difference_results_df_psma['Metric'].apply(lambda x: secondary_sort_key(x, secondary_order))
difference_results_df_psma = difference_results_df_psma.sort_values(by=['primary_sort_key', 'secondary_sort_key']).drop(['primary_sort_key', 'secondary_sort_key'], axis=1)

In [397]:
difference_results_df_psma

,Metric,U Statistic,P Value
7,Max: g/ml_PSMA_SalivaryGland,60.0,0.366372
23,Min: g/ml_PSMA_SalivaryGland,60.0,0.366372
15,Mean: g/ml_PSMA_SalivaryGland,55.0,0.239189
31,Peak: g/ml_PSMA_SalivaryGland,59.0,0.337952
39,Std. dev: g/ml_PSMA_SalivaryGland,57.0,0.285549
3,Max: g/ml_PSMA_Liver,81.0,0.848050
19,Min: g/ml_PSMA_Liver,82.0,0.805373
11,Mean: g/ml_PSMA_Liver,77.5,1.000000
27,Peak: g/ml_PSMA_Liver,77.5,1.000000
35,Std. dev: g/ml_PSMA_Liver,67.5,0.621416


### 3. Result for PSMA

Through t-tests, we found no metrics with significant differences between the FBTP and NFBTP groups (all p-values > 0.05). 

## II. Choline imaging parameters analysis

In [398]:
choline_filtered_metrics = ['Max', 'Min', 'Mean', 'Peak', 'Std. dev']
choline_cols = [col for col in data_con_lab.columns if ('Choline' in col and any(metric in col for metric in choline_filtered_metrics)) or col in ['FBTP/NFBTP',]]

Choline_imaging = data_con_lab[choline_cols]

### 1. Normal distribution test

In [399]:
normality_result_df_choline = check_normality(Choline_imaging)

normality_result_df_choline['primary_sort_key'] = normality_result_df_choline['Metric'].apply(lambda x: primary_sort_key(x, primary_order))
normality_result_df_choline['secondary_sort_key'] = normality_result_df_choline['Metric'].apply(lambda x: secondary_sort_key(x, secondary_order))

normality_result_df_choline = normality_result_df_choline.sort_values(by=['primary_sort_key', 'secondary_sort_key']).drop(columns=['primary_sort_key', 'secondary_sort_key'])

In [400]:
normality_result_df_choline

,Metric,Mean,Median,Min,Max,Normality p-value,Is Normal?
6,Max: g/ml_Choline_SalivaryGland,6.1748,6.45,3.07,9.16,0.592538,Yes
20,Min: g/ml_Choline_SalivaryGland,2.6012,2.72,1.29,3.86,0.583210,Yes
13,Mean: g/ml_Choline_SalivaryGland,3.9520,3.99,1.96,6.11,0.653871,Yes
27,Peak: g/ml_Choline_SalivaryGland,4.9308,5.10,2.46,7.83,0.497554,Yes
34,Std. dev: g/ml_Choline_SalivaryGland,0.7992,0.83,0.40,1.25,0.642930,Yes
3,Max: g/ml_Choline_Liver,11.9600,11.95,8.87,16.93,0.381592,Yes
17,Min: g/ml_Choline_Liver,5.3912,5.29,3.81,8.82,0.139243,Yes
10,Mean: g/ml_Choline_Liver,8.7528,8.46,6.29,14.40,0.005421,No
24,Peak: g/ml_Choline_Liver,10.4384,10.50,7.75,16.15,0.038587,No
31,Std. dev: g/ml_Choline_Liver,1.0552,1.03,0.62,1.77,0.298462,Yes


### 2. Significant differences test

In [401]:
difference_results_df_choline = mann_whitney_test(Choline_imaging)
difference_results_df_choline['primary_sort_key'] = difference_results_df_choline['Metric'].apply(lambda x: primary_sort_key(x, primary_order))
difference_results_df_choline['secondary_sort_key'] = difference_results_df_choline['Metric'].apply(lambda x: secondary_sort_key(x, secondary_order))
difference_results_df_choline = difference_results_df_choline.sort_values(by=['primary_sort_key', 'secondary_sort_key']).drop(['primary_sort_key', 'secondary_sort_key'], axis=1)

In [402]:
difference_results_df_choline

,Metric,U Statistic,P Value
6,Max: g/ml_Choline_SalivaryGland,51.5,0.171035
20,Min: g/ml_Choline_SalivaryGland,51.0,0.162635
13,Mean: g/ml_Choline_SalivaryGland,60.0,0.366187
27,Peak: g/ml_Choline_SalivaryGland,61.0,0.396134
34,Std. dev: g/ml_Choline_SalivaryGland,60.0,0.366095
3,Max: g/ml_Choline_Liver,87.0,0.602941
17,Min: g/ml_Choline_Liver,109.0,0.084624
10,Mean: g/ml_Choline_Liver,99.0,0.239189
24,Peak: g/ml_Choline_Liver,96.0,0.311072
31,Std. dev: g/ml_Choline_Liver,64.5,0.511058


### 3. Result for Choline

In this analysis, we first conducted t-tests for each continuous variable between the FBTP and NFBTP groups and filtered out those with p-values less than 0.05. To validate the results of the t-tests, we performed normality tests on these variables. The following are the significant results that meet the conditions:

**Kidney and Bone-** , **Liver**


| Metric                      | t-test p-value | Normality | Remarks                 |
|-----------------------------|----------------|-----------|-------------------------|
| Min: g/ml_Choline_Liver     | 0.046342       | Yes       | Part of Liver values    |
| Max: g/ml_Choline_Kidney    | 0.018877       | Yes       | All Kidney values       |
| Min: g/ml_Choline_Kidney    | 0.019271       | Yes       | All Kidney values       |
| Mean: g/ml_Choline_Kidney   | 0.023121       | Yes       | All Kidney values       |
| Peak: g/ml_Choline_Kidney   | 0.009228       | Yes       | All Kidney values       |
| Std. dev: g/ml_Choline_Kidney | 0.019935     | Yes       | All Kidney values       |
| Max: g/ml_Choline_Bone-     | 0.024791       | Yes       | All Bone- values        |
| Min: g/ml_Choline_Bone-     | 0.024786       | Yes       | All Bone- values        |
| Mean: g/ml_Choline_Bone-    | 0.029555       | Yes       | All Bone- values        |
| Peak: g/ml_Choline_Bone-    | 0.043033       | Yes       | All Bone- values        |
| Std. dev: g/ml_Choline_Bone-| 0.029333       | No        | Part of Bone- values    |

Among these, **Std. dev: g/ml_Choline_Bone-** showed significant differences in the t-test but did not meet the normality assumption.

# Biological parameters

## I. Clinical Assessment Parameters
- AGE
- Gleason
- WHO
- TNM

### 1. AGE

In [403]:
# age
age_cols = [col for col in data_con_lab.columns if col in ['FBTP/NFBTP','AGE']]
age = data_con_lab[age_cols]
normality_result_df_age = check_normality(age)
difference_results_df_age = mann_whitney_test(age)

In [404]:
print(normality_result_df_age)
print(difference_results_df_age)

  Metric   Mean  Median   Min   Max  Normality p-value Is Normal?
0    AGE  72.12    74.0  50.0  85.0            0.24816        Yes
  Metric  U Statistic   P Value
0    AGE         64.0  0.492695


### 2. Gleason

In [405]:
Gleason_cols = [col for col in data_cat_lab.columns if col in ['FBTP/NFBTP','Gleason']]
gleason = data_cat_lab[Gleason_cols]
gleason = gleason[gleason['Gleason'] != 0]
difference_results_df_gleason = chi_square_test(gleason)

In [406]:
print(difference_results_df_gleason)

  Variable  Chi2 Statistic   P Value  Degrees of Freedom
0  Gleason        3.325519  0.504899                   4


### 3. WHO

In [407]:
WHO_cols = [col for col in data_cat_lab.columns if col in ['FBTP/NFBTP','WHO']]
who = data_cat_lab[WHO_cols]
difference_results_df_who = chi_square_test(who)

In [408]:
print(difference_results_df_who)

  Variable  Chi2 Statistic   P Value  Degrees of Freedom
0      WHO        3.016775  0.221266                   2


### 4. TNM - T_stage

In [409]:
TNM_cols = [col for col in data_cat_lab.columns if col in ['FBTP/NFBTP','T_stage']]
tnm = data_cat_lab[TNM_cols]

In [410]:
tnm = tnm.dropna(subset=['T_stage', 'FBTP/NFBTP'])

contingency_table = pd.crosstab(tnm['T_stage'], tnm['FBTP/NFBTP'])

chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

result = {
    'Test': 'Chi-Square Test',
    'Chi2 Statistic': chi2,
    'P Value': p,
    'Degrees of Freedom': dof,
}
print("Result:")
print(result)

Result:
{'Test': 'Chi-Square Test', 'Chi2 Statistic': 2.916666666666667, 'P Value': 0.40465279495160544, 'Degrees of Freedom': 3}


## II. Blood Count
- Hemoglobin
- Leukocytes
- Neutrophils
- Lymphocytes
- Platelets

In [411]:
# Blood Count
CBC_filtered_metrics = ['Hemoglobin', 'Leukocytes', 'Neutrophils', 'Lymphocytes', 'Platelets']
CBC_cols = [col for col in data_con_lab.columns if ( any(metric in col for metric in CBC_filtered_metrics)) or col in ['FBTP/NFBTP',]]
cbc = data_con_lab[CBC_cols]
normality_result_df_cbc = check_normality(cbc)
difference_results_df_cbc = mann_whitney_test(cbc)

In [412]:
print(normality_result_df_cbc)
print(difference_results_df_cbc)

                      Metric       Mean   Median     Min     Max  \
0  Pre-cure_Hemoglobin(g/dL)   11.98800   12.100    9.40   15.20   
1   Pre-cure_Leukocytes(G/L)    6.47000    5.900    3.60   13.60   
2  Pre-cure_Neutrophils(G/L)    4.13936    3.632    2.21   10.27   
3  Pre-cure_Lymphocytes(G/L)    1.38500    1.440    0.14    2.68   
4    Pre-cure_Platelets(G/L)  231.12000  233.000  139.00  393.00   

   Normality p-value Is Normal?  
0           0.892095        Yes  
1           0.000318         No  
2           0.000046         No  
3           0.968541        Yes  
4           0.183935        Yes  
                      Metric  U Statistic   P Value
0  Pre-cure_Hemoglobin(g/dL)         54.0  0.217507
1   Pre-cure_Leukocytes(G/L)        123.0  0.012742
2  Pre-cure_Lymphocytes(G/L)         95.0  0.338045
3  Pre-cure_Neutrophils(G/L)        115.0  0.040079
4    Pre-cure_Platelets(G/L)         98.5  0.249835


## II. Liver/Kidney function and PSA Level
- Liver
    - ASAT
    - ALAT
    - Total Bilirubin
    - Albumin
    - ALP

- Kidney
    - GFR
    - Creatinine

- PSA

### 1. Liver

In [413]:
# liver
Liver_cols = [col for col in data_con_lab.columns if 'Pre-cure_Liver' in col  or col in ['FBTP/NFBTP',]]
liver = data_con_lab[Liver_cols]
normality_result_df_liver = check_normality(liver)
difference_results_df_liver = mann_whitney_test(liver)

In [414]:
print(normality_result_df_liver)
print(difference_results_df_liver)

                                   Metric      Mean  Median   Min    Max  \
0                Pre-cure_Liver_ASAT(U/L)   30.6000    20.0  14.0  108.0   
1                Pre-cure_Liver_ALAT(U/L)   16.4800    12.0   0.0   81.0   
2  Pre-cure_Liver_Total_Bilirubin(µmol/L)    8.6644     8.5   4.0   15.5   
3             Pre-cure_Liver_Albumin(g/L)   38.2252    38.0  27.0   50.0   
4                 Pre-cure_Liver_ALP(U/L)  171.4000   111.0  48.0  514.0   

   Normality p-value Is Normal?  
0       2.150020e-05         No  
1       4.540279e-08         No  
2       7.122731e-01        Yes  
3       4.239915e-01        Yes  
4       2.354544e-02         No  
                                   Metric  U Statistic   P Value
0                Pre-cure_Liver_ALAT(U/L)         63.5  0.475641
1                 Pre-cure_Liver_ALP(U/L)        125.0  0.009258
2                Pre-cure_Liver_ASAT(U/L)         65.0  0.527546
3             Pre-cure_Liver_Albumin(g/L)         82.5  0.784053
4  Pre-cure_Li

### 2. Kidney


In [415]:
# kidney
Kidney_cols = [col for col in data_con_lab.columns if 'Pre-cure_Kidney' in col  or col in ['FBTP/NFBTP']]
kidney = data_con_lab[Kidney_cols]
normality_result_df_kidney = check_normality(kidney)
difference_results_df_kidney = mann_whitney_test(kidney)

In [416]:
print(normality_result_df_kidney)
print(difference_results_df_kidney)

                               Metric     Mean  Median   Min    Max  \
0         Pre-cure_Kidney_GFR(mL/min)  73.6736   83.84   0.0  118.0   
1  Pre-cure_Kidney_Creatinine(µmol/L)  88.5372   79.70  43.6  209.0   

   Normality p-value Is Normal?  
0           0.012495         No  
1           0.000017         No  
                               Metric  U Statistic   P Value
0  Pre-cure_Kidney_Creatinine(µmol/L)         84.0  0.721908
1         Pre-cure_Kidney_GFR(mL/min)         68.5  0.660558


### 3. PSA

In [417]:
# psa
PSA_cols = [col for col in data_con_lab.columns if 'PSA' in col  or col in ['FBTP/NFBTP']]
psa = data_con_lab[PSA_cols]
normality_result_df_psa = check_normality(psa)
difference_results_df_psa = mann_whitney_test(psa)

In [418]:
print(normality_result_df_psa)
print(difference_results_df_psa)

                Metric       Mean  Median  Min     Max  Normality p-value  \
0  Pre-cure_PSA(ng/mL)  162.85372    51.7  0.0  2009.0       1.261201e-12   

  Is Normal?  
0         No  
                Metric  U Statistic   P Value
0  Pre-cure_PSA(ng/mL)        102.0  0.175121


## III. Tumor Invasion
- Bone invasion
- Pelvic tissues
- Liver
- Lungs
- Lymph node

### 1. Bone invasion

In [419]:
# Bone invasion
Bone_filtered_metrics = ['Cranium', 'Cervical Spine', 'Thoracic Spine', 'Lumbar Spine', 'Sternum', 'Ribs', 'Scapula', 'Pelvis', 'Humerus', 'Femur']
Bone_cols = [col for col in data_con_lab.columns if ( '100' in col or any(metric in col for metric in Bone_filtered_metrics)) or col in ['FBTP/NFBTP',]]
bone = data_con_lab[Bone_cols]
normality_result_df_bone = check_normality(bone)
difference_results_df_bone = mann_whitney_test(bone)

In [420]:
print(normality_result_df_bone)
print(difference_results_df_bone)

                Metric   Mean  Median  Min   Max  Normality p-value Is Normal?
0   Cervical Spine_Inv   1.90     1.0  0.0   8.0           0.020081         No
1          Cranium_Inv   0.72     0.0  0.0   6.0           0.000001         No
2            Femur_Inv   2.24     1.0  0.0   9.0           0.090666        Yes
3          Humerus_Inv   1.44     0.0  0.0   8.0           0.000261         No
4     Lumbar Spine_Inv   3.24     2.0  0.0  10.0           0.108505        Yes
5           Pelvis_Inv   3.98     3.0  0.0  10.0           0.020435         No
6             Ribs_Inv   2.66     2.0  0.0   8.0           0.056387        Yes
7          Scapula_Inv   1.80     1.0  0.0   7.0           0.036962         No
8          Sternum_Inv   2.28     1.0  0.0  10.0           0.006239         No
9   Thoracic Spine_Inv   3.66     3.0  0.0  10.0           0.042107         No
10      Total /100_Inv  23.92    14.0  0.0  71.0           0.134979        Yes
                Metric  U Statistic   P Value
0   Ce

### 2. Pelvic tissues/ Liver / Lungs / Lymph node

In [421]:
Inv_filtered_metrics = ['NumLN', 'PTInv', 'Lung_Inv', 'Liver_Inv']
Inv_cols = [col for col in data_cat_lab.columns if ( any(metric in col for metric in Inv_filtered_metrics)) or col in ['FBTP/NFBTP',]]
inv = data_cat_lab[Inv_cols]
difference_results_df_inv = chi_square_test(inv)

In [422]:
print(difference_results_df_inv)

         Variable  Chi2 Statistic   P Value  Degrees of Freedom
0  SupraNumLN_Inv        6.990666  0.136382                   4
1    SubNumLN_Inv        6.302180  0.390205                   6
2           PTInv        0.049808  0.823397                   1
3        Lung_Inv        0.000000  1.000000                   1
4       Liver_Inv        1.033673  0.309298                   1


# Fixation and Discordance

In [423]:
data_cat_lab.columns

Index(['Gleason', 'WHO', 'SupraNumLN_Inv', 'SubNumLN_Inv', 'PTInv', 'Lung_Inv',
       'Liver_Inv', 'Fixation_PSMA', 'Fixation_choline', 'Fixation_FDG',
       'Discordance', 'Discordance_PSMA-choline', 'Discordance_PSMA-FDG',
       'Discordance_choline-FDG', 'Difference_PSMA-choline',
       'Difference_PSMA-FDG', 'Difference_choline-FDG', 'T_stage',
       'FBTP/NFBTP'],
      dtype='object')

In [424]:
Fix_filtered_metrics = ['Fixation', 'Discordance', 'Difference']
Fix_cols = [col for col in data_cat_lab.columns if ( any(metric in col for metric in Fix_filtered_metrics)) or col in ['FBTP/NFBTP',]]
fix = data_cat_lab[Fix_cols]
difference_results_df_fix = chi_square_test(fix)

In [425]:
difference_results_df_fix

,Variable,Chi2 Statistic,P Value,Degrees of Freedom
0,Fixation_PSMA,1.885331,0.389588,2
1,Fixation_choline,2.340368,0.310310,2
2,Fixation_FDG,0.000000,1.000000,1
3,Discordance,0.394381,0.530005,1
4,Discordance_PSMA-choline,0.631010,0.426985,1
5,Discordance_PSMA-FDG,0.677083,0.410593,1
6,Discordance_choline-FDG,2.170770,0.140656,1
7,Difference_PSMA-choline,5.968808,0.050570,2
8,Difference_PSMA-FDG,7.366667,0.025139,2
9,Difference_choline-FDG,2.400000,0.301194,2


## 